In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the datasets
# /content/drive/MyDrive/Data/activities.csv
users_details_df = pd.read_csv('/content/drive/MyDrive/Data/user_details.csv')
trips_df = pd.read_csv('/content/drive/MyDrive/Data/trips.csv')
legs_df = pd.read_csv('/content/drive/MyDrive/Data/legs.csv')
leg_coordinates_df = pd.read_csv('/content/drive/MyDrive/Data/legs_coordinates.csv')
activities_df = pd.read_csv('/content/drive/MyDrive/Data/activities.csv')
experience_factors_df = pd.read_csv('/content/drive/MyDrive/Data/experience_factors.csv')
purposes_df = pd.read_csv('/content/drive/MyDrive/Data/purposes.csv')
weather_legs_df = pd.read_csv('/content/drive/MyDrive/Data/weather_legs.csv')
user_generic_df = pd.read_csv('/content/drive/MyDrive/Data/user_generic_worthwhileness_values.csv')
user_specific_df = pd.read_csv('/content/drive/MyDrive/Data/user_specific_worthwhileness_values.csv')
mots_df = pd.read_csv('/content/drive/MyDrive/Data/mots.csv')
weather_raw_df = pd.read_csv('/content/drive/MyDrive/Data/weather_raw.csv')
worthwhileness_df = pd.read_csv('/content/drive/MyDrive/Data/worthwhileness_elements_from_trips.csv')

In [ ]:
# Creating a list of dataframes and their corresponding names
dfs = [users_details_df, trips_df, legs_df, leg_coordinates_df, activities_df,
       experience_factors_df, purposes_df, weather_legs_df, user_generic_df,
       user_specific_df, mots_df, weather_raw_df, worthwhileness_df]

df_names = ['users_details_df', 'trips_df', 'legs_df', 'leg_coordinates_df',
            'activities_df', 'experience_factors_df', 'purposes_df',
            'weather_legs_df', 'user_generic_df', 'user_specific_df',
            'mots_df', 'weather_raw_df', 'worthwhileness_df']

# Display DataFrame names and their columns
for df, name in zip(dfs, df_names):
    print(f"DataFrame Name: {name}")
    print("Columns:", ", ".join(df.columns))
    print("-"*80)  # just a line for separation


In [79]:
import pandas as pd

# Assuming 'rating' is a column in your original DataFrame
positive_experience = experience_factors_df[experience_factors_df['plus'] == True]

In [ ]:
experience_factors_df.columns

In [ ]:
positive_experience.columns

In [83]:
user_activities = activities_df.groupby('legid')['activity'].apply(list).reset_index()
user_experience = positive_experience.groupby('legid').agg({
    'factor': list,
}).reset_index()

user_data = user_activities.merge(user_experience, on='legid', how='inner')


In [ ]:
user_data.columns

In [85]:
def compute_interest_profile(data):
    profile = {}
    for activities in data['activity']:
        for activity in activities:
            if activity in profile:
                profile[activity] += 1
            else:
                profile[activity] = 1

    for factors in data['factor']:
        for factor in factors:
            if factor in profile:
                profile[factor] += 1
            else:
                profile[factor] = 1

    return profile

user_profiles = user_data.groupby('legid').apply(compute_interest_profile)


In [86]:
def recommend_activities(user_profile, all_activities):
    recommendations = []
    for activity in all_activities:
        if activity not in user_profile:
            recommendations.append(activity)
    return recommendations[:5]  # returning top 5 activities not yet tried by the user

all_activities = activities_df['activity'].unique()
user_activity_recommendations = {legid: recommend_activities(profile, all_activities) for legid, profile in user_profiles.items()}


In [ ]:
!pip install surprise

In [ ]:
experience_factors_df.columns

In [88]:
experience_factors_df

,legid,factor,type,minus,plus
0,#24:3498,Ability_To_Do_What_I_Wanted,ACT,0,1
1,#25:3400,Simplicity_Difficulty_Of_The_Route,GT,0,1
2,#25:3460,Ability_To_Do_What_I_Wanted,ACT,0,1
3,#24:3479,Ability_To_Do_What_I_Wanted,ACT,0,1
4,#22:5269,Vehicle_Quality,WYR,0,1
...,...,...,...,...,...
377267,#223:47077,Simplicity_Difficulty_Of_The_Route,GT,0,1
377268,#223:47077,Ability_To_Do_What_I_Wanted,ACT,0,1
377269,#223:47077,Todays_Weather,CP,0,1
377270,#225:47060,Ability_To_Do_What_I_Wanted,ACT,0,1


In [ ]:
user_details = users_details_df.merge(user_specific_df,on ='userid',how='right')
user_details.shape

In [ ]:
# Join relevant dataframes
merged_df = legs_df.merge(users_details_df, on='userid', how='left').drop_duplicates()
merged_df.shape

In [ ]:
experience_factors_df.shape

In [ ]:
merged_df1 = merged_df.merge(experience_factors_df, on='legid', how='left').drop_duplicates()
merged_df1

In [ ]:
weather_legs_df.shape

In [ ]:
merged_df2 = merged_df1.merge(weather_legs_df[['legid', 'weather_scenario', 'apparent_temperature', 'wind_description']], on='legid', how='left')
merged_df2.shape

In [ ]:
experience_factors_df.columns

In [ ]:
merged_df2.columns

In [ ]:
nan_columns = merged_df2.columns[merged_df2.isna().any()].tolist()
print(nan_columns)


In [102]:
from sklearn.impute import KNNImputer

# Example for 'true_distance'
imputer = KNNImputer(n_neighbors=5)  # you can vary the number of neighbors
merged_df2['true_distance'] = imputer.fit_transform(merged_df2[['true_distance']])
merged_df2['leg_distance'] = imputer.fit_transform(merged_df2[['leg_distance']])

In [103]:
merged_df2 = merged_df2.drop(columns=['registration_date','apparent_temperature'])

In [104]:
merged_df2.to_csv('/content/drive/MyDrive/Data/merged_data.csv',index=False)

In [107]:
merged_df2 = pd.read_csv('/content/drive/MyDrive/Data/merged_data1.csv')

In [110]:
merged_df2 = pd.read_csv('/content/drive/MyDrive/Data/merged_data.csv')

In [ ]:
nan_columns = merged_df2.columns[merged_df2.isna().any()].tolist()
print(nan_columns)

In [ ]:
merged_df2['years_of_residence_household'].unique()

In [75]:
merged_df2.to_csv('/content/drive/MyDrive/Data/merged_data.csv',index=False)

In [ ]:
merged_df2.dtypes

In [113]:
from sklearn.impute import SimpleImputer

# For the 'plus' column
plus_imputer = SimpleImputer(strategy='most_frequent')
merged_df2['plus'] = plus_imputer.fit_transform(merged_df2[['plus']])

# For categorical columns
categorical_cols = ['transport_category', 'lang', 'education_level', 'marital_status_household',
                    'labour_status_household', 'factor', 'type', 'weather_scenario', 'wind_description']

cat_imputer = SimpleImputer(strategy='most_frequent')
merged_df2[categorical_cols] = cat_imputer.fit_transform(merged_df2[categorical_cols])

# For numerical columns like 'true_distance', 'leg_distance'
numerical_cols = ['true_distance', 'leg_distance']

num_imputer = SimpleImputer(strategy='median')
merged_df2[numerical_cols] = num_imputer.fit_transform(merged_df2[numerical_cols])


In [114]:
from sklearn.impute import SimpleImputer

# For the 'minus' column
minus_imputer = SimpleImputer(strategy='most_frequent')
merged_df2['minus'] = minus_imputer.fit_transform(merged_df2[['minus']])

# For categorical columns
categorical_cols = ['transport_category', 'lang', 'education_level', 'marital_status_household',
                    'labour_status_household', 'factor', 'type', 'weather_scenario', 'wind_description']

cat_imputer = SimpleImputer(strategy='most_frequent')
merged_df2[categorical_cols] = cat_imputer.fit_transform(merged_df2[categorical_cols])

# For numerical columns like 'true_distance', 'leg_distance'
numerical_cols = ['true_distance', 'leg_distance']

num_imputer = SimpleImputer(strategy='median')
merged_df2[numerical_cols] = num_imputer.fit_transform(merged_df2[numerical_cols])


In [115]:
from sklearn.impute import KNNImputer

# Define columns to impute
ordinal_cols = ['number_people_household']

for col in ordinal_cols:
    merged_df2[col] = merged_df2[col].replace('5+', 5).astype(float)

# Now run KNN imputation
imputer = KNNImputer(n_neighbors=5)
merged_df2[ordinal_cols] = imputer.fit_transform(merged_df2[ordinal_cols])

# Convert back if necessary
# (only if you want to retain the '5+' representation)
for col in ordinal_cols:
    merged_df2[col] = merged_df2[col].replace(5, '5+')


In [121]:
merged_df2.to_csv('/content/drive/MyDrive/Data/merged_data.csv',index=False)

In [4]:
nan_columns = merged_df2.columns[merged_df2.isna().any()].tolist()
print(nan_columns)

[]


In [119]:
from sklearn.impute import SimpleImputer

# Mapping categories to numbers
mapping = {
    'Less than 1': 1,
    '1 to 5': 2,
    '5+': 3,  # assuming that '5+' means from 5 to "More than 5"
    'More than 5': 4
}

# Replace categories with numbers
merged_df2['years_of_residence_encoded'] = merged_df2['years_of_residence_household'].map(mapping)

# Using SimpleImputer to fill NaNs
imputer = SimpleImputer(strategy='most_frequent')
merged_df2['years_of_residence_encoded'] = imputer.fit_transform(merged_df2[['years_of_residence_encoded']])

# Mapping numbers back to categories
inverse_mapping = {v: k for k, v in mapping.items()}
merged_df2['years_of_residence_household'] = merged_df2['years_of_residence_encoded'].map(inverse_mapping)

# Drop the encoded column
merged_df2.drop('years_of_residence_encoded', axis=1, inplace=True)


In [7]:
merged_df2.to_csv('/content/drive/MyDrive/Data/merged_data.csv',index=False)

In [ ]:
merged_df2.columns

In [1]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Data/merged_data.csv')

In [ ]:
print(legs_df.columns)
print(legs_df.shape)

In [14]:
trips_df = trips_df.drop(columns=['start_date','end_date'])

In [ ]:
df1 = trips_df.merge(users_details_df,on='userid')
df1.shape

In [ ]:
df2 = legs_df.merge(df1,on=['userid','tripid'])
df2.shape

In [18]:
df2 = df2.drop(columns=['use_trip_more_for','registration_date','lang'])

In [ ]:
df2.isna().sum()

In [21]:
from sklearn.impute import KNNImputer

# Example for 'true_distance'
imputer = KNNImputer(n_neighbors=5)  # you can vary the number of neighbors
df2['true_distance'] = imputer.fit_transform(df2[['true_distance']])
df2['leg_distance'] = imputer.fit_transform(df2[['leg_distance']])

In [22]:
from sklearn.impute import SimpleImputer

# Mapping categories to numbers
mapping = {
    'Less than 1': 1,
    '1 to 5': 2,
    '5+': 3,  # assuming that '5+' means from 5 to "More than 5"
    'More than 5': 4
}

# Replace categories with numbers
df2['years_of_residence_encoded'] = df2['years_of_residence_household'].map(mapping)

# Using SimpleImputer to fill NaNs
imputer = SimpleImputer(strategy='most_frequent')
df2['years_of_residence_encoded'] = imputer.fit_transform(df2[['years_of_residence_encoded']])

# Mapping numbers back to categories
inverse_mapping = {v: k for k, v in mapping.items()}
df2['years_of_residence_household'] = df2['years_of_residence_encoded'].map(inverse_mapping)

# Drop the encoded column
df2.drop('years_of_residence_encoded', axis=1, inplace=True)


In [23]:
from sklearn.impute import KNNImputer

# Define columns to impute
ordinal_cols = ['number_people_household']

for col in ordinal_cols:
    df2[col] = df2[col].replace('5+', 5).astype(float)

# Now run KNN imputation
imputer = KNNImputer(n_neighbors=5)
df2[ordinal_cols] = imputer.fit_transform(df2[ordinal_cols])

# Convert back if necessary
# (only if you want to retain the '5+' representation)
for col in ordinal_cols:
    df2[col] = df2[col].replace(5, '5+')


In [25]:
import random

def random_sample_imputation(data, column):
    """Impute missing values with a random sample."""
    # Extract non-missing values from the column
    non_missing = data[column].dropna().values
    # Create a Series of randomly sampled values (of the same length as the missing values)
    random_sample = random.choices(non_missing, k=data[column].isna().sum())
    # Replace NaN values with the random sample
    data.loc[data[column].isna(), column] = random_sample
    return data

# List of categorical columns with missing values
categorical_columns = ['transport_category', 'education_level', 'marital_status_household', 'labour_status_household']

# Apply random sample imputation to each column
for col in categorical_columns:
    df2 = random_sample_imputation(df2, col)


In [ ]:
df2.isna().sum()

In [27]:
df2.to_csv('/content/drive/MyDrive/Data/mood_data.csv',index=False)